In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

trades = pd.DataFrame(pd.read_csv("historical_data.csv"))
sentiment = pd.DataFrame(pd.read_csv("fear_greed_index.csv"))



Part A

In [139]:
print("Trader Shape:", trades.shape)
print("Sentiment Shape:", sentiment.shape)

Trader Shape: (211224, 16)
Sentiment Shape: (2644, 4)


In [140]:
print(trades.head())
print(sentiment.head())


                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   

   Closed PnL                                   Transaction Hash     Order ID  \
0         0.0  0xec0945

In [141]:
trades = trades.rename(columns={
    "Timestamp IST": "timestamp",
    "Closed PnL": "pnl",
    "Size USD": "size_usd"
})

In [142]:
trades["timestamp"] = pd.to_datetime(
    trades["timestamp"],
    format="%d-%m-%Y %H:%M",
    errors="coerce"
)

trades["date"] = trades["timestamp"].dt.date
trades["date"] = pd.to_datetime(trades["date"])

In [143]:
trades = trades[trades["pnl"] != 0]

In [144]:
sentiment["date"] = pd.to_datetime(
    sentiment["timestamp"],
    unit="s"
)

sentiment["date"] = sentiment["date"].dt.date
sentiment["date"] = pd.to_datetime(sentiment["date"])

sentiment = sentiment[["date", "classification"]]

In [145]:
print("Trader Date Range:",
      trades["date"].min(),
      "to",
      trades["date"].max())

print("Sentiment Date Range:",
      sentiment["date"].min(),
      "to",
      sentiment["date"].max())

Trader Date Range: 2023-12-14 00:00:00 to 2025-05-01 00:00:00
Sentiment Date Range: 2018-02-01 00:00:00 to 2025-05-02 00:00:00


In [146]:
trades["win"] = trades["pnl"] > 0
daily = trades.groupby(["Account", "date"]).agg(
    daily_pnl=("pnl", "sum"),
    num_trades=("pnl", "count"),
    avg_size=("size_usd", "mean"),
    win_rate=("win", "mean"),
    avg_fee=("Fee", "mean")
).reset_index()

print(daily.head())

                                      Account       date      daily_pnl  \
0  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-22  -21227.000000   
1  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-26    1603.100000   
2  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-28 -132271.000000   
3  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-12-06 -175611.000056   
4  0x083384f897ee0f19899168e3b1bec365f52a9012 2025-01-19   71393.964098   

   num_trades      avg_size  win_rate   avg_fee  
0          12  28186.666667  0.000000  2.818666  
1          12  19350.834167  1.000000  6.772791  
2          82  16058.536341  0.000000  1.605853  
3          33  36736.362424  0.000000  3.673636  
4         112  14162.418393  0.205357  3.423407  


In [147]:
daily = trades.groupby(["Account", "date"]).agg(
    daily_pnl=("pnl", "sum"),
    num_trades=("pnl", "count"),
    avg_size=("size_usd", "mean"),
    win_rate=("win", "mean"),
    avg_fee=("Fee", "mean")
).reset_index()

print(daily.head())

                                      Account       date      daily_pnl  \
0  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-22  -21227.000000   
1  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-26    1603.100000   
2  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-28 -132271.000000   
3  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-12-06 -175611.000056   
4  0x083384f897ee0f19899168e3b1bec365f52a9012 2025-01-19   71393.964098   

   num_trades      avg_size  win_rate   avg_fee  
0          12  28186.666667  0.000000  2.818666  
1          12  19350.834167  1.000000  6.772791  
2          82  16058.536341  0.000000  1.605853  
3          33  36736.362424  0.000000  3.673636  
4         112  14162.418393  0.205357  3.423407  


In [148]:
final = daily.merge(
    sentiment,
    on="date",
    how="left"
)

print("Missing sentiment rows:",
      final["classification"].isna().sum())

final = final.dropna(subset=["classification"])

Missing sentiment rows: 1


Part B — ANALYSIS

In [149]:
performance = final.groupby("classification")[[
    "daily_pnl", "win_rate"
]].mean()

print(performance)

                  daily_pnl  win_rate
classification                       
Extreme Fear    5819.765736  0.771535
Extreme Greed   6856.493209  0.867883
Fear            7544.169532  0.862488
Greed           4799.395699  0.846074
Neutral         4684.495201  0.835721


In [150]:
plt.figure(figsize=(8,5))
sns.boxplot(data=final, x="classification", y="daily_pnl")
plt.title("Daily PnL Distribution by Sentiment")
plt.show()

/var/folders/hx/zg31js_n0qg66j9l3d0jklnr0000gn/T/ipykernel_16048/1726918369.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [151]:
behavior = final.groupby("classification")[[
    "num_trades", "avg_size"
]].mean()

print(behavior)

                num_trades      avg_size
classification                          
Extreme Fear     81.937008  13233.385364
Extreme Greed    52.659091   6401.600105
Fear             66.984270  11124.925645
Greed            56.196429   7194.089772
Neutral          65.793478   8574.102957


In [152]:
volatility = final.groupby("classification")["daily_pnl"].std()
print(volatility)

classification
Extreme Fear     33071.542893
Extreme Greed    31516.105946
Fear             37459.746993
Greed            36716.519066
Neutral          20230.459513
Name: daily_pnl, dtype: float64
